In [18]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
#path = DATA_DIR/'sample_gtfs.zip'
path = DATA_DIR/'other_feeds'/'gtfs.zip'
#path = DATA_DIR/'other_feeds'/'seq_20170310_gtfs.zip'
feed = gt.read_gtfs(path, dist_units='km')

# Pick date
date = feed.get_first_week()[0]
print('date', date)


date 20170102


In [37]:
v = gt.time_it(feed.validate)()
v

Timing validate
2017-04-15 20:41:05.813320 Began process
2017-04-15 20:41:06.283989 Finished in 0.00 min


,message_type,message,table,rows
0,error,service_name is not a valid column name,calendar,[]
1,error,holiday_name is not a valid column name,calendar_dates,[]
2,error,agency_id is not a valid column name,fare_attributes,[]
3,error,feed_id is not a valid column name,feed_info,[]
4,error,feed_license is not a valid column name,feed_info,[]
5,error,feed_contact_email is not a valid column name,feed_info,[]
6,error,feed_contact_url is not a valid column name,feed_info,[]
7,error,route_sort_order is not a valid column name,routes,[]
8,error,min_headway_minutes is not a valid column name,routes,[]
9,error,platform_code is not a valid column name,stops,[]


In [38]:
f = feed.stop_times.copy()
f

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,605529A5070B5817,06:00:00,06:00:00,759499,1,NaN,0,0,0.000000
1,605529A5070B5817,NaN,NaN,759764,2,NaN,0,0,0.153227
2,605529A5070B5817,NaN,NaN,759901,3,NaN,0,0,0.311479
3,605529A5070B5817,NaN,NaN,759865,4,NaN,0,0,0.405412
4,605529A5070B5817,NaN,NaN,759866,5,NaN,0,0,0.583028
5,605529A5070B5817,06:04:00,06:04:00,759867,6,NaN,0,0,0.719317
6,605529A5070B5817,NaN,NaN,759868,7,NaN,0,0,0.848097
7,605529A5070B5817,NaN,NaN,759869,8,NaN,0,0,0.966317
8,605529A5070B5817,NaN,NaN,759870,9,NaN,0,0,1.184102
9,605529A5070B5817,NaN,NaN,759871,10,NaN,0,0,1.308977


In [21]:
feed.agency.columns

Index(['agency_url', 'agency_name', 'agency_timezone'], dtype='object')

In [ ]:
feed.describe()

In [ ]:
report = feed.validate()

In [ ]:
report

In [ ]:
f = feed.stop_times.copy()
i = 10
f.ix[9:11]

In [ ]:
feed.assess()

In [ ]:
ts = feed.compute_trip_stats()
#ts.to_csv(str(DATA_DIR/'cairns_trip_stats.csv'), index=False)
ts.head()

In [ ]:
rts = feed.compute_route_time_series(ts, date)
gt.downsample(rts, freq='12H')

In [ ]:
trip_id = feed.trips['trip_id'].iat[0]
geo = feed.trip_to_geojson(trip_id, include_stops=True)
path = DATA_DIR/'trip_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [ ]:
route_id = feed.routes['route_id'].iat[0]
geo = feed.route_to_geojson(route_id, include_stops=True)
path = DATA_DIR/'route_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [ ]:
# Add distances to feed
#
trip_stats = feed.compute_trip_stats(compute_dist_from_shapes=True)
feed = feed.append_dist_to_stop_times(trip_stats)
feed.stop_times.T

In [ ]:
trip_stats = feed.compute_trip_stats()
route_stats = feed.compute_route_stats(trip_stats, date)
route_stats


In [ ]:
a = pd.Series([np.nan, np.nan])
a.dropna().max()
#gt.get_trips(feed, date, "07:30:00")


In [ ]:

# Pick screen line
path = DATA_DIR/'cairns_screen_line.geojson'
with path.open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])

g = feed.compute_screen_line_counts(line, date)
g

In [ ]:
g.groupby(['route_id', 'orientation']).count()